# Notable Narratives and content

# collective repository
- GitHub:   https://github.com/ModelSEED/MicrobiomeNotebooks.git

# utilized Narratives
- MAG genomes: https://narrative.kbase.us/narrative/155805 & https://narrative.kbase.us/narrative/163264
- Pangenome-enhanced genomes: https://narrative.kbase.us/narrative/187797
- Annotated pangenome-enhanced genomes: https://narrative.kbase.us/narrative/188040
- Reconstructed MAG models: https://narrative.kbase.us/narrative/186678

# previously-computed files
- FAA of representative sequences: /scratch/fliu/data/cliff/mmseqs_ani_prob_rep_genome.faa
- Cluster genome mapping: /scratch/fliu/data/cliff/mmseqs_ani_prob.json

In [16]:
genome_ws = [155805, 163264]
mag_ws = 188065
new_workspace = 187471
pangenome_mag_ws = 187040
prob_mapping = "/scratch/fliu/data/cliff/mmseqs_ani_prob_v2.json"
prob_rep_mapping = "/scratch/fliu/data/cliff/mmseqs_ani_prob_rep_genome_v2.faa"

# Loading hit data from json file

In [17]:
%run cliffcommutil.py
with open("/scratch/fliu/data/cliff/mmseqs_ani_prob_v2.json", 'r') as f:
    hit_data = json.load(f)

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/


# Parsing hits for close genome list and establishing thresholds

In [18]:
%run cliffcommutil.py
mag_ani_data = {}
mag_thresholds = {}
mag_genomes = {}
mag_hit_distribution = {}
max_bins = 27
min_sim = 74
for protein in hit_data:
    for mag in hit_data[protein]:
        if mag not in mag_ani_data:
            mag_ani_data[mag] = {}
            mag_hit_distribution[mag] = [0] * max_bins
        for hit in hit_data[protein][mag]:
            genome = hit.split(":")[0]
            if hit_data[protein][mag][hit] < 100 and hit_data[protein][mag][hit] >= min_sim:
                if genome not in mag_ani_data[mag]:
                    mag_hit_distribution[mag][int(hit_data[protein][mag][hit]-min_sim)] += 1
                mag_ani_data[mag][genome] = hit_data[protein][mag][hit]
util.save("mag_ani_data",mag_ani_data)
util.save("mag_hit_distribution",mag_hit_distribution)
records = []
for mag in mag_hit_distribution:
    if mag not in mag_thresholds:
        mag_thresholds[mag] = {"threshold":0,"threshold_count":0}
    record = {"mag":mag}
    threshold = 0
    for i in range(max_bins):
        record["ani_" + str(min_sim+i)] = mag_hit_distribution[mag][i]
    threshold_count = 0
    for i in range(max_bins):
        index = max_bins-1-i  
        if mag_hit_distribution[mag][index] > 0 and (threshold == 0 or threshold == min_sim+index+1) and threshold_count <= 4 and (threshold_count == 0 or mag_hit_distribution[mag][index]/threshold_count < 5):
            threshold_count += mag_hit_distribution[mag][index]
            threshold = min_sim+index
    mag_thresholds[mag]["threshold"] = threshold
    mag_thresholds[mag]["threshold_count"] = threshold_count
    if threshold_count == 1:
        new_threshold = 0
        additional_count = 0
        start = max_bins-(threshold-min_sim)
        for i in range(start,max_bins,1):
            index = max_bins-1-i
            if mag_hit_distribution[mag][index] > 0 and (new_threshold == 0 or new_threshold == min_sim+index+1) and additional_count <= 4 and (additional_count == 0 or mag_hit_distribution[mag][index]/additional_count < 5):
                additional_count += mag_hit_distribution[mag][index]
                new_threshold = min_sim+index
        if new_threshold != 0:
            record["new_threshold"] = new_threshold
            record["new_threshold_count"] = additional_count
            if (threshold-new_threshold)<=10 or threshold != 100:
                mag_thresholds[mag]["threshold"] = new_threshold
                mag_thresholds[mag]["threshold_count"] = threshold_count+additional_count
    record["threshold"] = threshold
    record["threshold_count"] = threshold_count
    records.append(record)
for mag in mag_ani_data:
    mag_genomes[mag] = {}
    for genome in mag_ani_data[mag]:
        if mag_ani_data[mag][genome] >= mag_thresholds[mag]["threshold"]:
            mag_genomes[mag][genome] = mag_ani_data[mag][genome]
df = pd.DataFrame.from_records(records)
df.to_csv(util.output_dir+"/mag_hit_distribution.csv",index=False)
util.save("mag_thresholds",mag_thresholds)
util.save("mag_genomes",mag_genomes)

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/


# Assigning thresholds manually (not needed)

In [19]:
%run cliffcommutil.py
mag_thresholds = util.load("mag_thresholds")
mag_ani_data = util.load("mag_ani_data")
mag_genomes = {}
for mag in mag_ani_data:
    mag_genomes[mag] = {}
    for genome in mag_ani_data[mag]:
        if mag_ani_data[mag][genome] >= mag_thresholds[mag]["threshold"]:
            mag_genomes[mag][genome] = mag_ani_data[mag][genome]
    mag_thresholds[mag]["threshold_count"] = len(mag_genomes[mag])

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/


# Getting workspace IDs for MAGs

In [20]:
%run cliffcommutil.py
mag_wsids = {}
for ws in genome_ws:
    for mag in util.msrecon.kbase_api.list_objects(ws, object_type="KBaseGenomes.Genome", include_metadata=False):
        mag_wsids[mag[1]] = mag[7]
util.save("mag_wsids",mag_wsids)
util.save("mag_list",mag_list)

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/


# Identifying supplemental proteins for MAGs based on thresholds and hits

In [23]:
%run cliffcommutil.py
mag_protein_supplements = {}
mag_thresholds = util.load("mag_thresholds")
mag_genomes = util.load("mag_genomes")
mag_list = util.load("mag_list")
mag_wsids = util.load("mag_wsids")
magNames = util.load("mag_to_name")
all_mag_deep_data = {}
for magname in mag_wsids:
    all_mag_deep_data[magname] = {}
for protein in hit_data:
    for mag in hit_data[protein]:
        magname = magNames[mag]+".pg.G.D"
        if mag_thresholds[mag]["threshold"] > 0:
            nr_mags_found = {}
            rejected_mags_found = {}
            genomes_found = {}
            all_mag_deep_data[magname][protein] = {
                "self":False,
                "total":[0,0],
                "nr_mags":[0,0],
                "rejected_mags":[0,0],
                "genomes":[0,0]
            }
            for hit in hit_data[protein][mag]:
                (genome,gene) = hit.split(":")
                if genome == "self":
                    all_mag_deep_data[magname][protein]["self"] = True
                elif genome in mag_list:
                    if genome not in nr_mags_found:
                        all_mag_deep_data[magname][protein]["total"][0] += 1
                        all_mag_deep_data[magname][protein]["nr_mags"][0] += 1
                        if hit_data[protein][mag][hit] >= mag_thresholds[mag]["threshold"]:
                            if genome not in mag_genomes[mag]:
                                print(genome+":"+mag+":"+str(hit_data[protein][mag][hit])+"/"+str(mag_thresholds[mag]["threshold"]))
                            all_mag_deep_data[magname][protein]["total"][1] += 1
                            all_mag_deep_data[magname][protein]["nr_mags"][1] += 1
                        nr_mags_found[genome] = hit_data[protein][mag][hit]
                    elif nr_mags_found[genome] != hit_data[protein][mag][hit]:
                        print("Warning: different similarity values for the same genome")
                elif genome in mag_wsids:
                    if genome not in rejected_mags_found:
                        all_mag_deep_data[magname][protein]["total"][0] += 1
                        all_mag_deep_data[magname][protein]["rejected_mags"][0] += 1
                        if hit_data[protein][mag][hit] >= mag_thresholds[mag]["threshold"]:
                            if genome not in mag_genomes[mag]:
                                print(genome+":"+mag+":"+str(hit_data[protein][mag][hit])+"/"+str(mag_thresholds[mag]["threshold"]))
                            all_mag_deep_data[magname][protein]["total"][1] += 1
                            all_mag_deep_data[magname][protein]["rejected_mags"][1] += 1
                        rejected_mags_found[genome] = hit_data[protein][mag][hit]
                    elif rejected_mags_found[genome] != hit_data[protein][mag][hit]:
                        print("Warning: different similarity values for the same genome")
                else:
                    if genome not in genomes_found:
                        all_mag_deep_data[magname][protein]["total"][0] += 1
                        all_mag_deep_data[magname][protein]["genomes"][0] += 1
                        if hit_data[protein][mag][hit] >= mag_thresholds[mag]["threshold"]:
                            if genome not in mag_genomes[mag]:
                                print(genome+":"+mag+":"+str(hit_data[protein][mag][hit])+"/"+str(mag_thresholds[mag]["threshold"]))
                            all_mag_deep_data[magname][protein]["total"][1] += 1
                            all_mag_deep_data[magname][protein]["genomes"][1] += 1
                        genomes_found[genome] = hit_data[protein][mag][hit]
                    elif genomes_found[genome] != hit_data[protein][mag][hit]:
                        print("Warning: different similarity values for the same genome")
            #We only add supplemental proteins for a family that does not already have a protein in the MAG
            if not all_mag_deep_data[magname][protein]["self"]:
                if mag not in mag_protein_supplements:
                    mag_protein_supplements[mag] = {}
                mag_protein_supplements[mag][protein] = [all_mag_deep_data[magname][protein]["total"][1],all_mag_deep_data[magname][protein]["nr_mags"][1],all_mag_deep_data[magname][protein]["rejected_mags"][1]]
for magname in all_mag_deep_data:
    util.save("deepdata/"+mag,all_mag_deep_data[magname])
util.save("mag_protein_supplements",mag_protein_supplements)

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/


KeyboardInterrupt: 

# Load functional data (not needed)

In [8]:
%run cliffcommutil.py
function_hit_data = json.load(open('/home/fliu/cliff_mags/data/annotation_ani_prob_lo_85.json'))
mag_function_supplements = {}
mag_thresholds = util.load("mag_thresholds")
mag_list = util.load("mag_list")
mag_wsids = util.load("mag_wsids")
all_func_mag_deep_data = {}
for mag in mag_wsids:
    all_func_mag_deep_data[mag] = {}
for protein in function_hit_data:
    for mag in function_hit_data[protein]:
        nr_mags_found = {}
        rejected_mags_found = {}
        genomes_found = {}
        all_func_mag_deep_data[mag][protein] = {
            "self":False,
            "total":[0,0],
            "nr_mags":[0,0],
            "rejected_mags":[0,0],
            "genomes":[0,0]
        }
        for hit in function_hit_data[protein][mag]:
            (genome,gene) = hit.split(":")
            if genome == "self":
                all_func_mag_deep_data[mag][protein]["self"] = True
            elif genome in mag_list:
                if genome not in nr_mags_found:
                    all_func_mag_deep_data[mag][protein]["total"][0] += 1
                    all_func_mag_deep_data[mag][protein]["nr_mags"][0] += 1
                    if function_hit_data[protein][mag][hit] >= mag_thresholds[mag]["threshold"]/100:
                        all_func_mag_deep_data[mag][protein]["total"][1] += 1
                        all_func_mag_deep_data[mag][protein]["nr_mags"][1] += 1
                    nr_mags_found[genome] = function_hit_data[protein][mag][hit]
                elif nr_mags_found[genome] != function_hit_data[protein][mag][hit]:
                    print("Warning: different similarity values for the same genome")
            elif genome in mag_wsids:
                if genome not in rejected_mags_found:
                    all_func_mag_deep_data[mag][protein]["total"][0] += 1
                    all_func_mag_deep_data[mag][protein]["rejected_mags"][0] += 1
                    if function_hit_data[protein][mag][hit] >= mag_thresholds[mag]["threshold"]/100:
                        all_func_mag_deep_data[mag][protein]["total"][1] += 1
                        all_func_mag_deep_data[mag][protein]["rejected_mags"][1] += 1
                    rejected_mags_found[genome] = function_hit_data[protein][mag][hit]
                elif rejected_mags_found[genome] != function_hit_data[protein][mag][hit]:
                    print("Warning: different similarity values for the same genome")
            else:
                if genome not in genomes_found:
                    all_func_mag_deep_data[mag][protein]["total"][0] += 1
                    all_func_mag_deep_data[mag][protein]["genomes"][0] += 1
                    if function_hit_data[protein][mag][hit] >= mag_thresholds[mag]["threshold"]/100:
                        all_func_mag_deep_data[mag][protein]["total"][1] += 1
                        all_func_mag_deep_data[mag][protein]["genomes"][1] += 1
                    genomes_found[genome] = function_hit_data[protein][mag][hit]
                elif genomes_found[genome] != function_hit_data[protein][mag][hit]:
                    print("Warning: different similarity values for the same genome")
        #We only add supplemental proteins for a family that does not already have a protein in the MAG
        if not all_func_mag_deep_data[mag][protein]["self"]:
            if mag not in mag_function_supplements:
                mag_function_supplements[mag] = {}
            mag_function_supplements[mag][protein] = [all_func_mag_deep_data[mag][protein]["total"][1],all_func_mag_deep_data[mag][protein]["nr_mags"][1],all_func_mag_deep_data[mag][protein]["rejected_mags"][1]]
for mag in mag_function_supplements:
    util.save("funcdeepdata/"+mag,all_func_mag_deep_data[mag])
util.save("mag_function_supplements",mag_function_supplements)

Output files printed to:/scratch/chenry/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/kb_sdk_home/run_local/workdir/tmp/


# Computing stats on additional proteins

In [27]:
%run cliffcommutil.py
mag_list = util.load("mag_list")
mag_thresholds = util.load("mag_thresholds")
#mag_protein_supplements = util.load("mag_protein_supplements")
records = []
mag_probability_threshold = {}
for mag in mag_list:
    if mag[1] in mag_protein_supplements:
        count = mag_thresholds[mag[1]]["threshold_count"]
        record = {"mag":mag[1]}
        records.append(record)
        abundance_count = [0] * 400
        for protein in mag_protein_supplements[mag[1]]:
            fraction = mag_protein_supplements[mag[1]][protein][0]/count
            entry = int(fraction*100/5)
            abundance_count[entry] += 1
        total = 0
        final_total = None
        mag_probability_threshold[mag[1]] = 0.1
        for (i,entry) in enumerate(abundance_count):
            index = len(abundance_count)-i-1
            lasttotal = total
            total += abundance_count[index]
            if total >= 20000 and final_total == None:
                threshold = 5*(index+1)/100
                if threshold >= 0.1:
                    mag_probability_threshold[mag[1]] = threshold
                    final_total = lasttotal
            if index == 2 and final_total == None:
                final_total = total
        print(mag[1],mag_probability_threshold[mag[1]],final_total)
        for (i,entry) in enumerate(abundance_count):
            record[i] = entry
util.save("mag_probability_threshold",mag_probability_threshold)
df = pd.DataFrame.from_records(records)
df.to_csv(util.output_dir+"/protein_count_abundance.csv",index=False)

Salt_Pond_MetaGSF2_B_H2O_MG_DASTool_bins_metabat.8.contigs__.RAST 0.1 14737
Salt_Pond_MetaG_R1_B_D1_MG_DASTool_bins_metabat.20.contigs__.RAST 0.1 3860
Salt_Pond_MetaG_R2_B_D2_MG_DASTool_bins_concoct_out.4.contigs__.RAST 0.1 4004
Salt_Pond_MetaG_R1_A_D2_MG_DASTool_bins_concoct_out.21.contigs__.RAST 0.1 4192
Salt_Pond_MetaG_R1_C_D1_MG_DASTool_bins_metabat.31.contigs__.RAST 0.1 5106
Salt_Pond_MetaG_R2_A_H2O_MG_DASTool_bins_concoct_out.29.contigs__.RAST 0.1 3670
Salt_Pond_MetaG_R2_B_D2_MG_DASTool_bins_metabat.50.contigs__.RAST 0.1 4271
Salt_Pond_MetaG_R1_B_D2_MG_DASTool_bins_metabat.18.contigs__.RAST 0.1 4144
Salt_Pond_MetaG_R1_A_D1_MG_DASTool_bins.metabat.15.contigs__.RAST 0.1 1457
Salt_Pond_MetaG_R2_C_D1_MG_DASTool_bins_concoct_out.88.contigs__.RAST 0.1 2330
Salt_Pond_MetaG_R2_restored_C_black_MG_DASTool_bins_concoct_out.57.contigs__.RAST 0.1 1738
Salt_Pond_MetaG_R2_A_D1_MG_DASTool_bins_metabat.10.contigs__.RAST 0.1 2713
Salt_Pond_MetaGSF2_B_D1_MG_DASTool_bins_concoct_out.44.contigs__.RA

# Renaming MAGs

In [3]:
%run cliffcommutil.py
mag_to_name = {}
name_to_mag = {}
mag_list = util.load("mag_list")
for mag in mag_list:
    original_name = mag[1]
    new_name = original_name
    new_name = new_name.replace("Salt_Pond_MetaG_", "", 1)
    new_name = new_name.replace("Salt_Pond_MetaG", "", 1)
    new_name = new_name.replace("_MG_DASTool_bins_", "", 1)
    new_name = new_name.replace("metabat", "", 1)
    new_name = new_name.replace("concoct_out", "", 1)
    new_name = new_name.replace("maxbin", "", 1)
    new_name = new_name.replace(".contigs__.RAST", "", 1)
    mag_to_name[mag[1]] = new_name
    name_to_mag[new_name] = mag[1]
    print(name_to_mag)
util.save("mag_to_name",mag_to_name)
util.save("name_to_mag",name_to_mag)

Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir


1722057900.422139 ERROR: Requested data mag_list doesn't exist at /Users/chenry/code/notebooks/MicrobiomeNotebooks/Cliff/datacache/mag_list.json


ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


ValueError: Requested data mag_list doesn't exist at /Users/chenry/code/notebooks/MicrobiomeNotebooks/Cliff/datacache/mag_list.json

# Loading protein sequence data

In [14]:
%run cliffcommutil.py
from Bio import SeqIO
protein_hash = {}
for record in SeqIO.parse('/scratch/fliu/data/cliff/mmseqs_ani_prob_rep_genome_v2.faa', 'fasta'):
    protein_hash[record.id] = record.seq

Output files printed to:/scratch/chenry/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/kb_sdk_home/run_local/workdir/tmp/


# Building genomes and assemblies

In [28]:
%run cliffcommutil.py
unfiltered_list = util.load("mag_list")
filter = [
  "Salt_Pond_MetaG_R2_restored_C_black_MG_DASTool_bins_concoct_out.17.contigs__.RAST",
  "Salt_Pond_MetaG_R2_A_D1_MG_DASTool_bins_concoct_out.52.contigs__.RAST",
  "Salt_Pond_MetaG_R2_restored_DShore_MG_DASTool_bins_concoct_out.38.contigs__.RAST",
  "Salt_Pond_MetaG_R2A_C_D2_MG_DASTool_bins_metabat.27.contigs__.RAST",
  "Salt_Pond_MetaG_R1_A_D2_MG_DASTool_bins_concoct_out.25.contigs__.RAST",
  "Salt_Pond_MetaG_R2_B_D1_MG_DASTool_bins_concoct_out.110.contigs__.RAST",
  "Salt_Pond_MetaG_R1_A_D1_MG_DASTool_bins.metabat.47.contigs__.RAST",
  "Salt_Pond_MetaG_R2A_A_D2_MG_DASTool_bins_concoct_out.10.contigs__.RAST",
  "Salt_Pond_MetaGSF2_A_D2_MG_DASTool_bins_concoct_out.55.contigs__.RAST",
  "Salt_Pond_MetaG_R2_A_D1_MG_DASTool_bins_metabat.32.contigs__.RAST"
]
mag_list = []
for item in unfiltered_list:
    if item[1] in filter or len(filter) == 0:
        mag_list.append(item)
name_to_mag = util.load("name_to_mag")
mag_thresholds = util.load("mag_thresholds")
mag_probability_threshold = util.load("mag_probability_threshold")
#mag_protein_supplements = util.load("mag_protein_supplements")
for item in mag_list:
#for name in problem_list:
    full_data = util.get_object(item[1],item[7])
    genome_obj = full_data["data"]
    genome_obj["assembly_ref"] = str(item[6])+"/"+str(item[0])+"/"+str(item[4])+";"+genome_obj["assembly_ref"]
    name = mag_to_name[item[1]]
    cds_hash = {}
    for i,ftr in enumerate(genome_obj["cdss"]):
        cds_hash[ftr["id"]] = ftr
    ftr_hash = {}
    for i,ftr in enumerate(genome_obj["features"]):
        ftr_hash[ftr["id"]] = ftr
        ftr["id"] = name+"_"+str(i+1)
        if "cdss" in ftr:
            for j,cds in enumerate(ftr["cdss"]):
                cds_hash[cds]["id"] = ftr["id"]+".CDS"
                ftr["cdss"][j] = cds_hash[cds]["id"]
                cds_hash[cds]["parent_gene"] = ftr["id"]
    firstgene = genome_obj["features"][0]
    count = 0
    if item[1] in mag_protein_supplements:
        count = 1
        total_genomes = mag_thresholds[item[1]]["threshold_count"]
        for protein in mag_protein_supplements[item[1]]:
            if mag_protein_supplements[item[1]][protein][0]/total_genomes >= mag_probability_threshold[item[1]]:
                ftrid = name+'.pg_'+str(count)
                count += 1
                protseq = str(protein_hash[protein])
                dnaseq = util.translate_protein_to_gene(protseq)
                result = hashlib.md5(protseq.encode())
                md5 = result.hexdigest()
                result = hashlib.md5(dnaseq.encode())
                dnamd5 = result.hexdigest()
                newftr = {
                    "aliases": [["MMseqMD5",protein]],
                    "cdss": [
                        ftrid+".CDS"
                    ],
                    "functions":["Hypothetical protein"],
                    "dna_sequence": dnaseq,
                    "dna_sequence_length": 3*len(protseq),
                    "id": ftrid,
                    "location": [
                        [
                            firstgene["location"][0][0],
                            1,
                            "+",
                            3*len(protseq)
                        ]
                    ],
                    "md5": dnamd5,
                    "ontology_terms": {},
                    "protein_md5": md5,
                    "protein_translation": protseq,
                    "protein_translation_length": len(protseq),
                    "warnings": []
                }
                cdsftr = newftr.copy()
                del cdsftr["cdss"]
                cdsftr["id"] = ftrid+".CDS"
                cdsftr["parent_gene"] = ftrid
                genome_obj["features"].append(newftr)
                genome_obj["cdss"].append(cdsftr)
        print(item[1],count)
        #Saving MAG
        util.save("genome/"+name,full_data)

Output files printed to:/scratch/chenry/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/kb_sdk_home/run_local/workdir/tmp/
Salt_Pond_MetaGSF2_B_H2O_MG_DASTool_bins_metabat.8.contigs__.RAST 14738
Salt_Pond_MetaG_R1_B_D1_MG_DASTool_bins_metabat.20.contigs__.RAST 3861
Salt_Pond_MetaG_R2_B_D2_MG_DASTool_bins_concoct_out.4.contigs__.RAST 4005
Salt_Pond_MetaG_R2_restored_DShore_MG_DASTool_bins_concoct_out.46.contigs__.RAST 4005
Salt_Pond_MetaG_R1_A_D2_MG_DASTool_bins_concoct_out.21.contigs__.RAST 4193
Salt_Pond_MetaG_R1_C_D1_MG_DASTool_bins_metabat.31.contigs__.RAST 5107
Salt_Pond_MetaG_R2_A_H2O_MG_DASTool_bins_concoct_out.29.contigs__.RAST 3671
Salt_Pond_MetaG_R2_B_D2_MG_DASTool_bins_metabat.50.contigs__.RAST 4272
Salt_Pond_MetaG_R1_B_D2_MG_DASTool_bins_metabat.18.contigs__.RAST 4145
Salt_Pond_MetaGSF2_C_D2_MG_DASTool_bins_concoct_out.32.contigs__.RAST 4145
Salt_Pond_MetaG_R1_A_D1_MG_DASTool_bins.metabat.15.contigs__.RAST 1458
Salt_Po

# Finding and fixing redundant names

In [ ]:
%run cliffcommutil.py
mag_list = util.load("mag_list")
mag_to_name = util.load("mag_to_name")
name_to_mag = {}
for item in mag_list:
    if item[1] not in mag_to_name:
        print("Missing:",item[1])
    elif mag_to_name[item[1]] in name_to_mag:
        print("Collision:",mag_to_name[item[1]])
    else:
        name_to_mag[mag_to_name[item[1]]] = item[1]
util.save("name_to_mag",name_to_mag)

# Loading genomes to KBase

In [ ]:
%run cliffcommutil.py
unfiltered_list = util.load("mag_list")
filter = []
mag_list = []
for item in unfiltered_list:
    if item[1] in filter or len(filter) == 0:
        mag_list.append(item)
mag_to_name = util.load("mag_to_name")
from datetime import datetime
now = datetime.now()
timestamp = datetime.timestamp(now)
workspace = 187797
anno = util.anno_client()
anno.clients["GenomeFileUtil"] = util.gfu_client()
finished = util.load("finished_genomes",[])
unconverted = {}
for item in mag_list:
    name = mag_to_name[item[1]]
    if name not in finished:
        genome = util.load("annotated/"+name)
        count = 0
        for ftr in genome["features"]:
            end = len(name)+3
            if ftr["id"][0:4] == "Salt":
                unconverted[item[1]] = 1
            if ftr["id"][0:end] == name+".pg":
                count += 1
        print(name,count)
        util.save_ws_object(name+".pg",187797,genome,"KBaseGenomes.Genome")
        finished.append(name)
        util.save("finished_genomes",finished)
for item in unconverted:
    print(item)

# Inspecting genomes before load

In [ ]:
%run cliffcommutil.py
mags = util.msrecon.kbase_api.list_objects(187875, object_type="KBaseGenomes.Genome", include_metadata=False)
mag_hash = {}
for item in mags:
    mag_hash[item[1]] = item
mag_list = util.load("mag_list")
mag_to_name = util.load("mag_to_name")
for item in mag_list:
    name = mag_to_name[item[1]]
    name += ".pangenome.GLM4EC"
    if name in mag_hash:
        print("Getting "+name)
        full_data = util.get_object(name,187875)
        genome = full_data["data"]
        ftrhash = {}
        for ftr in genome["features"]:
            ftrhash[ftr["id"]] = ftr
        for ftr in genome["cdss"]:
            ftrhash[ftr["id"]] = ftr
            if "parent_gene" in ftr:
                if ftr["parent_gene"] not in ftrhash:
                    print(item[1],new_name,"Gene "+ftr["parent_gene"]+" not found!")
        for ftr in genome["features"]:
            if "cdss" in ftr:
                for cds in ftr["cdss"]:
                    if cds not in ftrhash:
                        print(item[1],new_name,"CDS "+cds+" not found!")

# Annotating genomes using RAST API

In [6]:
%run cliffcommutil.py
mag_list = util.load("mag_list")
mag_to_name = util.load("mag_to_name")
from modelseedpy.core.rpcclient import RPCClient
client = RPCClient("https://tutorial.theseed.org/services/genome_annotation")
for item in mag_list:
    name = mag_to_name[item[1]]
    if name not in output:
        rast_genome = {
            "id":item[1],
            "genetic_code":11,
            "scientific_name":"Unknown",
            "domain":"Bacteria",
            "contigs": [],
            "features":[]
        }
        data = util.load("genome/"+name)
        genome = data["data"]
        for ftr in genome["features"]:
            rast_genome["features"].append(ftr)
        workflow = {
            "stages":[
                {
                    "name": "annotate_proteins_kmer_v2", 
                    "kmer_v2_parameters": {}},
                {
                    "name": "annotate_proteins_kmer_v1",
                    "kmer_v1_parameters": {"annotate_hypothetical_only": 1},},
                {
                    "name": "annotate_proteins_similarity",
                    "similarity_parameters": {"annotate_hypothetical_only": 1},
                },
            ]
        }
        params = [{"features": rast_genome["features"]}, workflow]
        output = client.call("GenomeAnnotation.run_pipeline",params)
        util.save("rast/"+name,output)

Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


1722232442.262796 ERROR: Requested data genome/R1_B_D1.20 doesn't exist at /Users/chenry/code/notebooks/MicrobiomeNotebooks/Cliff/datacache/genome/R1_B_D1.20.json


<Response [200]>


ValueError: Requested data genome/R1_B_D1.20 doesn't exist at /Users/chenry/code/notebooks/MicrobiomeNotebooks/Cliff/datacache/genome/R1_B_D1.20.json

# Adding RAST ontology event

In [ ]:
%run cliffcommutil.py
mag_list = util.load("mag_list")
mag_to_name = util.load("mag_to_name")
from datetime import datetime
now = datetime.now()
timestamp = datetime.timestamp(now)
for item in mag_list:
    name = mag_to_name[item[1]]
    data = util.load("genome/"+name)
    rast = util.load("rast/"+name)
    annoapi = util.anno_client(native_python_api=True)
    events = [{
        "ontology_id" : "SSO",
        "description" : "RAST Annotation API",
        "method_version" : "1.0",
        "method" : "annotate_genome",
        "ontology_terms": {},
        "timestamp":"2024-07-29T19:32:45"
    }]
    for ftr in rast[0]["features"]:
        if "function" in ftr:
            events[0]["ontology_terms"][ftr["id"]] = [{"term":ftr["function"]}]
    output = annoapi.add_annotation_ontology_events({
        "output_workspace":187797,
        "events":events,
        "overwrite_matching":True,
        "object":data["data"],
        "type":"KBaseGenomes.Genome",
        "input_ref":None,
        "input_workspace":None,
        "output_name":None,
        "save":0
    })
    util.save("annotated/"+name,output["object"])

# Printing feature probabilities

In [10]:
%run cliffcommutil.py
mag_list = util.load("mag_list")
mag_thresholds = util.load("mag_thresholds")
mag_to_name = util.load("mag_to_name")
mag_protein_supplements = util.load("mag_protein_supplements")
feature_probabilities = {}
for item in mag_list:
    name = mag_to_name[item[1]]
    data = util.load("annotated/"+name)
    feature_probabilities[item[1]] = {}
    ftrs = data["features"]
    total_genomes = mag_thresholds[item[1]]["threshold_count"]
    for ftr in ftrs:
        if ftr["id"].split(".")[-1][0:3] == "pg_":
            feature_probabilities[item[1]][ftr["id"]] = mag_protein_supplements[item[1]][ftr["aliases"][0][1]][0]/total_genomes
            if feature_probabilities[item[1]][ftr["id"]] > 1:
                feature_probabilities[item[1]][ftr["id"]] = 1
        else:
            feature_probabilities[item[1]][ftr["id"]] = 1
util.save("feature_probabilities",feature_probabilities)

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/


# Processing metabolite data

In [12]:
%run cliffcommutil.py
metabolite_hash = util.load("met_NameMSID")

from numpy import array
newMets = array(["2'-Deoxyuridine", '2-Oxoglutarate', '2-Oxoisocaproate', '3-Hydroxybutyrate', '3-Hydroxyisovalerate', '3-Methyl-2-oxovalerate', '4-Aminobutyrate', 'Acetate', 'Acetone', 'Alanine', 'Arginine', 'Aspartate', 'Benzoate', 'Betaine', 'Dimethylamine', 'Ethanol', 'Formate', 'Fructose', 'Fumarate', 'Glucose', 'Glutamate', 'Glycerol', 'Isobutyrate', 'Isoleucine', 'Isopropanol', 'Isovalerate', 'Lactate', 'Leucine', 'Maltose', 'Methanol', 'Methionine', 'Methylamine', 'Methylguanidine', 'N,N-Dimethylglycine', 'Phenylacetate', 'Phenylalanine', 'Proline', 'Propionate', 'Propylene glycol', 'Pyroglutamate', 'Succinate', 'Sucrose', 'Thymidine', 'Trehalose', 'Trimethylamine', 'Tryptophan', 'Tyrosine', 'Uracil', 'Uridine', 'Valine'])
print(set(newMets)-set(list(metabolite_hash.keys())))

metabolite_names = {cpdID: metName for metName, cpdID in metabolite_hash.items()}
util.save("metabolite_names",metabolite_names)
util.save("metabolites",list(metabolite_names.keys()))

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/
set()


# Computing SMIPPS

In [2]:
%run cliffcommutil.py

def smipps(arguments):
    # unpack the arguments
    (mag_list, mags_to_models, model_ws, metabolites, feature_probabilities,problemlist, auxo_media, gmm_base_media, aerobicity
    ) = arguments
    
    # print status
    from multiprocess import current_process
    pid = current_process().name
    print(pid)
    
    # define the phenotype sets
    uptake_phenoset = util.create_phenotypeset_from_compounds(
        metabolites,
        base_media=auxo_media,
        base_uptake=0,
        base_excretion=1000,
        global_atom_limits={},
        type="uptake"
    )
    excretion_phenoset = util.create_phenotypeset_from_compounds(
        metabolites,
        base_media=auxo_media,
        base_uptake=0,
        base_excretion=1000,
        global_atom_limits={},
        type="excretion"
    )
    growth_phenoset = util.create_phenotypeset_from_compounds(
        metabolites,
        base_media=gmm_base_media,
        base_uptake=0,
        base_excretion=1000,
        global_atom_limits={},
        type="growth"
    )
    phenosets = {"uptake":uptake_phenoset,"excretion":excretion_phenoset,"growth":growth_phenoset}
    
    # compute the SMIPP for each MAG
    for i,mag in enumerate(mag_list):
        print(mag)
        name = mag[1]
        if name not in probability_finished and name not in problemlist:
            mdlutl = util.msrecon.get_model(name+model_suffix,model_ws)
            reaction_probabilities[name] = {}
            for rxn in mdlutl.model.reactions:
                highest_prob = None
                for gene in rxn.genes:
                    if gene.id in feature_probabilities[mag[1]]:
                        if highest_prob == None or feature_probabilities[mag[1]][gene.id] > highest_prob:
                            highest_prob = feature_probabilities[mag[1]][gene.id]
                if highest_prob != None:
                    rxn.probability = highest_prob
                    reaction_probabilities[name][rxn.id] = highest_prob

            print(mdlutl.model.genome_ref)
            genome = util.msrecon.get_msgenome_from_ontology(mdlutl.model.genome_ref,native_python_api=True,output_ws=None)
            reaction_hash = genome.annoont.get_reaction_gene_hash(feature_type="gene")
            for rxn in reaction_hash:
                highest_prob = None
                for gene in reaction_hash[rxn]:
                    if gene in feature_probabilities[mag[1]]:
                        if highest_prob == None or feature_probabilities[mag[1]][gene] > highest_prob:
                            highest_prob = feature_probabilities[mag[1]][gene]
                if highest_prob != None and (rxn+"_c0" not in reaction_probabilities[name] or highest_prob >= reaction_probabilities[name][rxn+"_c0"]):
                    if rxn+"_c0" in mdlutl.model.reactions:
                        mdlutl.model.reactions.get_by_id(rxn+"_c0").probability = highest_prob
                    reaction_probabilities[name][rxn+"_c0"] = highest_prob

            filters = mdlutl.get_attributes("gf_filter")
            tests = mdlutl.get_atp_tests(core_template=util.msrecon.core_template,atp_media_filename=util.msrecon.module_dir+"/data/atp_medias.tsv",recompute=False)
            msgapfill = MSGapfill(
                mdlutl,
                [util.msrecon.get_template(mdlutl.model.template_ref)],
                [],
                tests,
                blacklist=[],
                default_target="bio1",
                minimum_obj=0.01,
                base_media=None,
                base_media_target_element=None
            )

            #Adding missing transporter for metabolites to gapfilling database
            for cpd in metabolites:
                if "EX_"+cpd+"_e0" not in msgapfill.gfmodelutl.model.reactions:
                    transport = msgapfill.gfmodelutl.add_transport_and_exchange_for_metabolite(cpd,direction="=",prefix="trans",override=False)

            coefficients = {}
            gf_penalties = msgapfill.gfpkgmgr.getpkg("GapfillingPkg").gapfilling_penalties
            gfrxn = 0
            probrxn = 0
            otherrxn = 0
            for reaction in msgapfill.gfmodelutl.model.reactions:
                if reaction.id in reaction_probabilities[name]:
                    probrxn += 2
                    coefficients[">"+reaction.id] = 1-reaction_probabilities[name][reaction.id]
                    coefficients["<"+reaction.id] = 1-reaction_probabilities[name][reaction.id]
                elif reaction.id in gf_penalties:
                    if "forward" in gf_penalties[reaction.id]:
                        gfrxn += 1
                        coefficients[">"+reaction.id] = 1+gf_penalties[reaction.id]["forward"]
                    else:
                        otherrxn += 1
                        coefficients[">"+reaction.id] = 0.95
                    if "reverse" in gf_penalties[reaction.id]:
                        gfrxn += 1
                        coefficients["<"+reaction.id] = 1+gf_penalties[reaction.id]["reverse"]
                    else:
                        otherrxn += 1
                        coefficients["<"+reaction.id] = 0.95
                else:
                    otherrxn += 2
                    coefficients[">"+reaction.id] = 0.95
                    coefficients["<"+reaction.id] = 0.95
            print(name,"GF:",gfrxn,"Prob:",probrxn,"Other:",otherrxn)

            # Create conditional logic for growth phenotypes if errors are raised here 
            msgapfill.prefilter(test_conditions=tests,growth_conditions=[],use_prior_filtering=True,base_filter_only=True)

            gf_phenotype_results[name] = {}
            for phenoid in phenosets:
                gf_phenotype_results[name][phenoid] = {}
                output = phenosets[phenoid].simulate_phenotypes(
                    msgapfill.gfmodelutl,
                    multiplier=2,
                    add_missing_exchanges=True,
                    save_fluxes=False,
                    save_reaction_list=True,
                    gapfill_negatives=False,
                    msgapfill=None,
                    test_conditions=None,
                    ignore_experimental_data=True,
                    flux_coefficients=coefficients
                )
                for index, row in output["details"].iterrows():
                    if "reactions" in output["data"][row["Phenotype"]]:
                        output["data"][row["Phenotype"]]["average_probability"] = 0
                        for rxn in output["data"][row["Phenotype"]]["reactions"]:
                            direction = rxn[0:1]
                            rxnid = rxn[1:]
                            if direction == ">":
                                if rxnid not in gf_penalties or "forward" not in gf_penalties[rxnid]:
                                    if rxnid in reaction_probabilities[name]:
                                        output["data"][row["Phenotype"]]["average_probability"] += reaction_probabilities[name][rxnid]
                                    else:
                                        output["data"][row["Phenotype"]]["average_probability"] += 0.05
                            elif direction == "<":
                                if rxnid not in gf_penalties or "reverse" not in gf_penalties[rxnid]:
                                    if rxnid in reaction_probabilities[name]:
                                        output["data"][row["Phenotype"]]["average_probability"] += reaction_probabilities[name][rxnid]
                                    else:
                                        output["data"][row["Phenotype"]]["average_probability"] += 0.05
                        output["data"][row["Phenotype"]]["average_probability"] = output["data"][row["Phenotype"]]["average_probability"]/len(output["data"][row["Phenotype"]]["reactions"])
                    gf_phenotype_results[name][phenoid][row["Phenotype"]] = output["data"][row["Phenotype"]]
            probability_finished.append(name)
            util.save("new_gf_phenotype_results_",gf_phenotype_results)
            util.save("reaction_probabilities",reaction_probabilities)
            util.save("probability_finished_",probability_finished)
        break
                
                
# define function parameters
mag_list = util.load("mag_list")
model_ws = 188065
model_suffix = ".pyr"
metabolites = util.load("metabolites")
feature_probabilities = util.load("feature_probabilities")
reaction_probabilities = util.load("reaction_probabilities",{})
gf_phenotype_results = util.load("new_gf_phenotype_results_",{})
probability_finished = util.load("probability_finished_",[])
problemlist = util.load("problemlist",[])
mag_to_name = util.load("mag_to_name")
mag_feature_probabilities = {mag_to_name[k]+".pg.G.D":v for k,v in feature_probabilities.items()}
# print(mag_feature_probabilities.keys())
for mag in mag_list:
    mag[1] = mag_to_name[mag[1]]+".pg.G.D"

auxo_media = util.msrecon.get_media(f"{model_ws}/AuxoMedia")
gmm_base_media = util.msrecon.get_media(f"{model_ws}/PyruateMinimalAerobic")


# run the parallelized code
parallelize = True
if parallelize:
    from multiprocess import Pool
    from os import cpu_count
    from numpy import array_split
    
    # the MAGs are partitioned into groups of 4, and a thread is created to handle each group 
    mag_lists = array_split(mag_list, int(len(mag_list)/4))
    pool = Pool(min(cpu_count(), len(mag_lists)))
    args = [(mag_list, mag_to_name, model_ws, metabolites, mag_feature_probabilities, problemlist, auxo_media, gmm_base_media, "aerobic")
           for mag_list in mag_lists]
    list_of_outputs = pool.map(smipps, args)
else:
    outputs = smipps((mag_list, mag_to_name, model_ws, metabolites, mag_feature_probabilities, problemlist, auxo_media, gmm_base_media, "aerobic"))

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/


ConnectionError: HTTPSConnectionPool(host='kbase.us', port=443): Max retries exceeded with url: /services/ws (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fe57732fdd0>: Failed to establish a new connection: [Errno 111] Connection refused'))

# Consolidating gapfilling phenotype data sets 

In [ ]:
outputs

In [ ]:
%run cliffcommutil.py
consolidated_gf_results = util.load("consolidated_gf_results",{})
reaction_probabilities = util.load("reaction_probabilities",{})
for i in range(0,8,1):
    phenotype_gf_results = util.load("gf_phenotype_results_"+str(i))
    part_reaction_probabilities = util.load("reaction_probabilities"+str(i))
    for name in phenotype_gf_results:
        consolidated_gf_results[name] = phenotype_gf_results[name]
         reaction_probabilities[name] = part_reaction_probabilities[name]
util.save("consolidated_gf_results",consolidated_gf_results)
util.save("reaction_probabilities",reaction_probabilities)

# Computing proper gapfill and filtering bad predictions

In [ ]:
%run cliffcommutil.py
consolidated_gf_results = util.load("consolidated_gf_results")
reaction_probabilities = util.load("reaction_probabilities")
mag_to_model = util.load("mag_to_model")
studies = ["uptake","excretion","growth"]
for name in consolidated_gf_results:
    modelname = mag_to_model(name)
    #Getting base model
    mdlutl = util.msrecon.get_model(modelname,186678)
    
    for study in studies:
        if study not in consolidated_gf_results[name]:
            continue
        for phenotype in consolidated_gf_results[name][study]:
            if "reactions" in consolidated_gf_results[name][study][phenotype]:
                consolidated_gf_results[name][study][phenotype]["new_probability"] = 0
                consolidated_gf_results[name][study][phenotype]["probrxn"] = 0
                consolidated_gf_results[name][study][phenotype]["otherrxn"] = 0
                consolidated_gf_results[name][study][phenotype]["origgfrxn"] = 0
                consolidated_gf_results[name][study][phenotype]["new_gf"] = 0
                consolidated_gf_results[name][study][phenotype]["new_gf_reactions"] = []
                found = False
                for rxn in consolidated_gf_results[name][study][phenotype]["reactions"]:
                    direction = rxn[0:1]
                    rxnid = rxn[1:]
                    if rxnid[0:3] == "EX_" and rxnid[3:11] == phenotype:
                        found = True
                    if rxnid not in mdlutl.model.reactions:
                        consolidated_gf_results[name][study][phenotype]["new_gf_reactions"].append(rxn)
                        consolidated_gf_results[name][study][phenotype]["new_gf"] += 1
                    elif direction == ">":
                        if mdlutl.model.reactions.get_by_id(rxnid).upper_bound <= 0:
                            consolidated_gf_results[name][study][phenotype]["new_gf_reactions"].append(rxn)
                            consolidated_gf_results[name][study][phenotype]["new_gf"] += 1
                        elif rxnid in reaction_probabilities[name]:
                            consolidated_gf_results[name][study][phenotype]["new_probability"] += reaction_probabilities[name][rxnid]
                            consolidated_gf_results[name][study][phenotype]["probrxn"] += 1
                        elif rxnid[0:3] != "bio" and rxnid[0:3] != "EX_" and rxnid[0:3] != "DM_" and len(mdlutl.model.reactions.get_by_id(rxnid).genes) == 0:
                            consolidated_gf_results[name][study][phenotype]["origgfrxn"] += 1
                        else:
                            consolidated_gf_results[name][study][phenotype]["otherrxn"] += 1
                            consolidated_gf_results[name][study][phenotype]["new_probability"] += 0.05  
                    elif direction == "<":
                        if mdlutl.model.reactions.get_by_id(rxnid).lower_bound >= 0:
                            consolidated_gf_results[name][study][phenotype]["new_gf_reactions"].append(rxn)
                            consolidated_gf_results[name][study][phenotype]["new_gf"] += 1
                        elif rxnid in reaction_probabilities[name]:
                            consolidated_gf_results[name][study][phenotype]["new_probability"] += reaction_probabilities[name][rxnid]
                            consolidated_gf_results[name][study][phenotype]["probrxn"] += 1
                        elif rxnid[0:3] != "bio" and rxnid[0:3] != "EX_" and rxnid[0:3] != "DM_" and len(mdlutl.model.reactions.get_by_id(rxnid).genes) == 0:
                            consolidated_gf_results[name][study][phenotype]["origgfrxn"] += 1
                        else:
                            consolidated_gf_results[name][study][phenotype]["otherrxn"] += 1
                            consolidated_gf_results[name][study][phenotype]["new_probability"] += 0.05  
                totalrxn = consolidated_gf_results[name][study][phenotype]["probrxn"]+consolidated_gf_results[name][study][phenotype]["new_gf"]+consolidated_gf_results[name][study][phenotype]["otherrxn"]
                consolidated_gf_results[name][study][phenotype]["new_probability"] = consolidated_gf_results[name][study][phenotype]["new_probability"]/totalrxn

                if not found:
                    consolidated_gf_results[name][study][phenotype]["original_objective_value"] = consolidated_gf_results[name][study][phenotype]["objective_value"]
                    consolidated_gf_results[name][study][phenotype]["objective_value"] = 0
                    consolidated_gf_results[name][study][phenotype]["class"] = "N"
                    consolidated_gf_results[name][study][phenotype]["positive"] = False
                    consolidated_gf_results[name][study][phenotype]["new_probability"] = 0
util.save("consolidated_gf_results",consolidated_gf_results)

# Printing matrices

In [ ]:
%run cliffcommutil.py
consolidated_gf_results = util.load("consolidated_gf_results")
metabolites = util.load("metabolites")
metabolite_names = util.load("metabolite_names")
mag_abundances = util.load("mag_abundances")
total_mag_abundances = util.load("total_mag_abundances")
records = {
    "uptake_prob":[{"Sample":"Name"}],
    "excretion_prob":[{"Sample":"Name"}],
    "growth_prob":[{"Sample":"Name"}],
}
types = ["uptake","excretion","growth"]
for met in metabolites:
    for record in records:
        records[record][0][met] = metabolite_names[met]
for sample in mag_abundances:
    new_record = {
        "uptake_prob":{"Sample":sample},
        "excretion_prob":{"Sample":sample},
        "growth_prob":{"Sample":sample}
    }
    for metabolite in metabolites:
        for record in new_record:
            new_record[record][metabolite] = 0
    for name in mag_abundances[sample]:
        for metabolite in metabolites:
            for currtype in types:
                if currtype in consolidated_gf_results[name]:
                    if metabolite in consolidated_gf_results[name][currtype] and "new_probability" in consolidated_gf_results[name][currtype][metabolite]:
                        new_record[currtype+"_prob"][metabolite] += mag_abundances[sample][asvset]/total_mag_abundances[sample]*consolidated_gf_results[name][currtype][metabolite]["new_probability"]
    for record in records:
        records[record].append(new_record[record])
#Printing interation matrices
for record in records:
    df = pd.DataFrame.from_records(records[record])
    df.to_csv(util.output_dir+"/Sample"+record+"Interactions.csv",index=False)

# Printing ASV data

In [ ]:
%run cliffcommutil.py
consolidated_gf_results = util.load("consolidated_gf_results")
metabolites = util.load("metabolites")
metabolite_names = util.load("metabolite_names")
mag_list = util.load("mag_list")
records = {
    "uptake_prob":[{"MAG":"Name"}],
    "excretion_prob":[{"MAG":"Name"}],
    "growth_prob":[{"MAG":"Name"}],
    "uptake_data":[{"MAG":"Name"}],
    "excretion_data":[{"MAG":"Name"}],
    "growth_data":[{"MAG":"Name"}],
}
types = ["uptake","excretion","growth"]
for met in metabolites:
    for record in records:
        records[record][0][met] = metabolite_names[met]
for mag in mag_list:
    name = mag[1]
    if name not in consolidated_gf_results:
        print("No data for ",name)
        continue
    new_record = {}
    for currtype in types:
        new_record[currtype+"_prob"] = {"MAG":name}
        new_record[currtype+"_data"] = {"MAG":name}
    for metabolite in metabolites:
        for currtype in types:
            if currtype in consolidated_gf_results[name]:
                if metabolite in consolidated_gf_results[name][currtype] and "new_probability" in consolidated_gf_results[name][currtype][metabolite]:
                    new_record[currtype+"_prob"][metabolite] = consolidated_gf_results[name][currtype][metabolite]["new_probability"]
                    new_record[currtype+"_data"][metabolite] = str(consolidated_gf_results[name][currtype][metabolite]["probrxn"])
                    new_record[currtype+"_data"][metabolite] += "/"+str(consolidated_gf_results[name][currtype][metabolite]["otherrxn"])
                    new_record[currtype+"_data"][metabolite] += "/"+str(consolidated_gf_results[name][currtype][metabolite]["origgfrxn"])
                    new_record[currtype+"_data"][metabolite] += "/"+str(consolidated_gf_results[name][currtype][metabolite]["new_gf"])
                    new_record[currtype+"_data"][metabolite] += "/"+str(consolidated_gf_results[name][currtype][metabolite]["gapfill_count"])
                    new_record[currtype+"_data"][metabolite] += "/"+str(consolidated_gf_results[name][currtype][metabolite]["reaction_count"])
    for record in records:
        records[record].append(new_record[record])
#Printing interation matrices
for record in records:
    df = pd.DataFrame.from_records(records[record])
    df.to_csv(util.output_dir+"/MAG"+record+"Interactions.csv",index=False)

# Gapfilling models based on gapfilling simulation results

In [ ]:
%run cliffcommutil.py
import numpy as np
consolidated_gf_results = util.load("consolidated_gf_results")
finished_models = util.load("finished_models",{})
metabolites = util.load("metabolites")
reactions_to_add = {}

for name in consolidated_gf_results:
    if name not in finished_models:
        mdlutl = util.msrecon.get_model(name+".ASV.mdl",181152)
        types = ["uptake","excretion","growth"]
        reactions_to_add[name] = {}
        for currtype in types:
            problist = []
            for metabolite in metabolites:
                if metabolite in consolidated_gf_results[name][currtype] and "new_probability" in consolidated_gf_results[name][currtype][metabolite]:
                    if consolidated_gf_results[name][currtype][metabolite]["new_probability"] > 0:
                        problist.append(consolidated_gf_results[name][currtype][metabolite]["new_probability"])
            if len(problist) > 0:
                ave = np.array(problist).mean()
                stdev = np.array(problist).std()
                for metabolite in metabolites:
                    if metabolite in consolidated_gf_results[name][currtype] and "new_probability" in consolidated_gf_results[name][currtype][metabolite]:
                        if consolidated_gf_results[name][currtype][metabolite]["new_probability"] > 0:
                            consolidated_gf_results[name][currtype][metabolite]["zscore"] = (consolidated_gf_results[name][currtype][metabolite]["new_probability"]-ave)/stdev
                            if consolidated_gf_results[name][currtype][metabolite]["zscore"] >= -1 and "reactions" in consolidated_gf_results[name][currtype][metabolite]:
                                for rxn in consolidated_gf_results[name][currtype][metabolite]["reactions"]:
                                    direction = rxn[0:1]
                                    rxnid = rxn[1:]
                                    if rxnid not in mdlutl.model.reactions:
                                        if rxnid not in reactions_to_add[name]:
                                            reactions_to_add[name][rxnid] = {}
                                        reactions_to_add[name][rxnid][direction] = consolidated_gf_results[name][currtype][metabolite]["zscore"]
                                    elif direction == ">":
                                        if mdlutl.model.reactions.get_by_id(rxnid).upper_bound <= 0:
                                            if rxnid not in reactions_to_add[name]:
                                                reactions_to_add[name][rxnid] = {}
                                            reactions_to_add[name][rxnid][direction] = consolidated_gf_results[name][currtype][metabolite]["zscore"]
                                    elif direction == "<":
                                        if mdlutl.model.reactions.get_by_id(rxnid).lower_bound >= 0:
                                            if rxnid not in reactions_to_add[name]:
                                                reactions_to_add[name][rxnid] = {}
                                            reactions_to_add[name][rxnid][direction] = consolidated_gf_results[name][currtype][metabolite]["zscore"]
        util.save("consolidated_gf_results",consolidated_gf_results)
        msgapfill = MSGapfill(
            mdlutl,
            [util.msrecon.get_template(mdlutl.model.template_ref)],
            [],
            [],
            blacklist=[],
            default_target="bio1",
            minimum_obj=0.01,
            base_media=None,
            base_media_target_element=None
        )
        for cpd in metabolites:
            if "EX_"+cpd+"_e0" not in msgapfill.gfmodelutl.model.reactions:
                transport = msgapfill.gfmodelutl.add_transport_and_exchange_for_metabolite(cpd,direction="=",prefix="trans",override=False)
        for rxnid in reactions_to_add[name]:
            if rxnid in msgapfill.gfmodel.reactions:
                rxn = msgapfill.gfmodel.reactions.get_by_id(rxnid)
                rxn = rxn.copy()
                mdlutl.model.add_reactions([rxn])
                rxn.upper_bound = 0
                rxn.lower_bound = 0
                if ">" in reactions_to_add[name][rxnid]:
                    rxn.upper_bound = 100
                if "<" in reactions_to_add[name][rxnid]:
                    rxn.lower_bound = -100
            else:
                print(name,"Missing reaction",rxnid,reactions_to_add[name][rxnid])
        util.msrecon.save_model(mdlutl,181152,name+".ASV.mdl.gf")
        finished_models[name] = 1
    util.save("finished_models",finished_models)
util.save("reactions_to_add",reactions_to_add)

# Building and saving community model

In [ ]:
%run cliffcommutil.py
import cobra
mag_list = util.load("mag_list")
mag_abundances = util.load("mag_abundances")
total_mag_abundances = util.load("total_mag_abundances")

for sample in mag_abundances:
    member_models = []
    member_names = []
    member_abundances = {}
    asv_hash = {}
    for mag in mag_abundances[sample]:
        if mag_abundances[sample][mag]/total_mag_abundances[sample] > 0.01:
            mdlutl = util.msrecon.get_model(name+".ASV.mdl.gf",181152)
            member_models.append(mdlutl.model)
            member_names.append(name)
            member_abundances[name] = mag_abundances[sample][mag]/total_mag_abundances[sample]
    comm_model = MSCommunity.build_from_species_models(
        member_models,
        mdlid=sample,
        name=sample,
        names=member_names,
        abundances=member_abundances
    )
    cobra.io.save_json_model(comm_model.model, "models/"+sample+'.json')

# Running community model

In [ ]:
%run cliffcommutil.py
procid = 0
metabolites = util.load("metabolites")
metabolomics_data = util.load("metabolomics_data")
feature_probabilities = util.load("feature_probabilities")
community_model_data = util.load("community_model_data")
from optlang.symbolics import Zero, add
import cobra

complete_media = util.msrecon.get_media("KBaseMedia/Complete")

model_list = ["RC-ABX_-1.5","RC-ABX_1.5","RC-ABX_4.0","RC-ABX_6.0","RC-ABX_9.0","RC-ABX_12.5"]
output = {}
rxn_record_hash = {}
records = [
    {"id":"max_growth"},
    {"id":"carbon_uptake"},
    {"id":"flux fitting objective"},
    {"id":"minimum probability objective"},
]
for modelid in model_list:
    output[modelid] = {}
    base_model = cobra.io.load_json_model("models/"+modelid+'.json')
    current_comm_model = MSCommunity(
        model=base_model,
        names=community_model_data[modelid]["names"]
    )
    #loadedmodel.solver = 'gurobi'
    dipeptide_exchanges = [
        "EX_cpd11591_e0",
        "EX_cpd11589_e0",
        "EX_cpd15605_e0",
        "EX_cpd11588_e0",
        "EX_cpd11583_e0",
        "EX_cpd11580_e0",
        "EX_cpd11593_e0",
        "EX_cpd11585_e0",
        "EX_cpd11586_e0",
        "EX_cpd15604_e0",
        "EX_cpd11581_e0",
        "EX_cpd01017_e0",
        "EX_cpd11590_e0",
        "EX_cpd11592_e0",
        "EX_cpd11584_e0",
        "EX_cpd00731_e0",
        "EX_cpd15603_e0",
        "EX_cpd11587_e0",
        "EX_cpd11582_e0",
        "EX_cpd15606_e0",
        "EX_cpd03424_e0",
        "EX_cpd00423_e0",
        "EX_cpd00080_e0",
        "EX_cpd02233_e0",
        "EX_cpd00355_e0",
        "EX_cpd00235_e0",
        "EX_cpd00079_e0",
        "EX_cpd01912_e0"
    ]
    reaction_probabilities = {}
    for rxn in base_model.reactions:
        highest_prob = None
        for gene in rxn.genes:
            if gene.id in feature_probabilities:
                if highest_prob == None or feature_probabilities[gene.id] > highest_prob:
                    highest_prob = feature_probabilities[gene.id]
        if highest_prob != None:
            rxn.probability = highest_prob
            reaction_probabilities[rxn.id] = highest_prob
        elif rxn.id[0:3] != "bio" and rxn.id[0:3] != "EX_" and rxn.id[0:3] != "DM_" and len(rxn.genes) == 0:
            reaction_probabilities[rxn.id] = 0
        else:
            reaction_probabilities[rxn.id] = 0.05
    min_prob = 0.05
    prob_exp = 1
    ex_weight = 1
    kinetics_coef = 750
    mdlutl = current_comm_model.mdlutl
    pkgmgr = MSPackageManager.get_pkg_mgr(mdlutl)
    #Setting media
    pkgmgr.getpkg("KBaseMediaPkg").build_package(complete_media)
    #Adding commkinetic constraints
    pkgmgr.getpkg("CommKineticPkg").build_package(kinetics_coef, current_comm_model)
    #Adding elemental uptake constraints
    pkgmgr.getpkg("ElementUptakePkg").build_package({"C": 300})
    for rxn in base_model.reactions:
        if "EX_" == rxn.id[0:3]:
            currrxn = current_comm_model.model.reactions.get_by_id(rxn.id)
            if rxn.id in dipeptide_exchanges:
                currrxn.lower_bound = 0
                currrxn.upper_bound = 0
            else:
                if currrxn.lower_bound < 0:
                    currrxn.lower_bound = -1000
                if currrxn.upper_bound > 0:
                    currrxn.upper_bound = 1000
    currrxn = current_comm_model.model.reactions.get_by_id("EX_cpd00007_e0")
    currrxn.lower_bound = -20
    #Maximize biomass production
    mdlutl.model.objective = mdlutl.model.problem.Objective(Zero, direction="max")
    mdlutl.model.objective.set_linear_coefficients({mdlutl.model.reactions.bio1.forward_variable: 1})
    first_solution = mdlutl.model.optimize()
    output[modelid]["max_growth"] = first_solution.objective_value
    output[modelid]["carbon_uptake"] = pkgmgr.getpkg("ElementUptakePkg").variables["elements"]["C"].primal
    records[0][modelid] = output[modelid]["max_growth"]
    records[1][modelid] = output[modelid]["carbon_uptake"]
    print(modelid+" growth:", output[modelid]["max_growth"])
    print(modelid+"carbon uptake:",output[modelid]["carbon_uptake"])
    
    with open(util.output_dir+"/"+modelid+"-growth.lp", "w") as out:
        out.write(str(mdlutl.model.solver))

    if str(output[modelid]["max_growth"]) == "nan":
        print("Skipping condition due to infeasibility", modelid)
        continue
    #Constraining to 70% of community biomass
    pkgmgr.getpkg("ObjConstPkg").build_package(output[modelid]["max_growth"] * 0.5, None)
    #Creating minimal probability objective
    coef = {}
    flux_fitting_target = {}
    for rxn in base_model.reactions:
        if "rxn" == rxn.id[0:3]:
            probability = reaction_probabilities[rxn.id]
            currrxn = mdlutl.model.reactions.get_by_id(rxn.id)
            coef.update(
                {
                    currrxn.forward_variable: max(
                        min_prob, (1 - float(probability) ** prob_exp)
                    )
                }
            )
            coef.update(
                {
                    currrxn.reverse_variable: max(
                        min_prob, (1 - float(probability) ** prob_exp)
                    )
                }
            )
        elif "EX_" == rxn.id[0:3]:
            currrxn = mdlutl.model.reactions.get_by_id(rxn.id)
            if rxn.id[3:11] in metabolomics_data:
                flux_fitting_target[rxn.id] = -1*output[modelid]["max_growth"]*metabolomics_data[rxn.id[3:11]][modelid]
            coef.update({currrxn.forward_variable: ex_weight})
            coef.update({currrxn.reverse_variable: ex_weight})

    #Solving the LP
    #pkgmgr.getpkg("FluxFittingPkg").build_package({
    #    "target_flux": flux_fitting_target,
    #    "totalflux": 0,
    #    "set_objective": 1,
    #    "default_rescaling": 0.1,
    #    "rescale_vfit_by_flux": True
    #})
    stuck_reactions = {}
    for iteration in range(1,11,1):
        print("Iteration",iteration)
        for rxn in flux_fitting_target:
            if rxn in stuck_reactions:
                continue
            currrxn = mdlutl.model.reactions.get_by_id(rxn)
            starting_point = first_solution.fluxes[rxn]
            distance = abs(flux_fitting_target[rxn] - starting_point)
            original_upper = currrxn.upper_bound
            original_lower = currrxn.lower_bound
            if starting_point > flux_fitting_target[rxn]:
                if starting_point-iteration*distance/10 < currrxn.upper_bound:
                    currrxn.lower_bound = starting_point-iteration*distance/10
                    currrxn.upper_bound = starting_point-iteration*distance/10
                else:
                    currrxn.upper_bound = starting_point-iteration*distance/10
                    currrxn.lower_bound = starting_point-iteration*distance/10
            else:
                if flux_fitting_target[rxn]-(10-iteration)*distance/10 < currrxn.upper_bound:
                    currrxn.lower_bound = flux_fitting_target[rxn]-(10-iteration)*distance/10
                    currrxn.upper_bound = flux_fitting_target[rxn]-(10-iteration)*distance/10
                else:
                    currrxn.upper_bound = flux_fitting_target[rxn]-(10-iteration)*distance/10
                    currrxn.lower_bound = flux_fitting_target[rxn]-(10-iteration)*distance/10
            solution = mdlutl.model.optimize()
            if solution.status != "optimal":
                print("Stuck",rxn,starting_point,flux_fitting_target[rxn],original_upper)
                stuck_reactions[rxn] = 1
                if original_upper >= currrxn.lower_bound:
                    currrxn.upper_bound = original_upper
                    currrxn.lower_bound = original_lower
                else:
                    currrxn.lower_bound = original_lower
                    currrxn.upper_bound = original_upper
    for rxn in flux_fitting_target:
        currrxn = mdlutl.model.reactions.get_by_id(rxn)
        print(currrxn.id,currrxn.lower_bound,currrxn.upper_bound)
    #solution = mdlutl.model.optimize()
    #output[modelid]["flux fitting objective"] = solution.objective_value
    
    #with open(util.output_dir+"/"+modelid+"-fitting.lp", "w") as out:
    #    out.write(str(mdlutl.model.solver))

    #records[2][modelid] = output[modelid]["flux fitting objective"]
    #for rxn in flux_fitting_target:
    #    currrxn = current_comm_model.model.reactions.get_by_id(rxn)
    #    if solution.fluxes[rxn] > 0:
    #        currrxn.upper_bound = solution.fluxes[rxn]
    #        currrxn.lower_bound = solution.fluxes[rxn]
    #    elif solution.fluxes[rxn] < 0:
    #        currrxn.lower_bound = solution.fluxes[rxn]
    #        currrxn.upper_bound = solution.fluxes[rxn]
    #Setting probability minimization objective
    mdlutl.model.objective = mdlutl.model.problem.Objective(Zero, direction="min")
    mdlutl.model.objective.set_linear_coefficients(coef)
    solution = mdlutl.model.optimize()
    if solution.status != "optimal":
        while solution.status != "optimal":
            print("Infeasible: reducing objective constraint",0.9*mdlutl.pkgmgr.getpkg("ObjConstPkg").constraints["objc"]["1"].lb)
            mdlutl.pkgmgr.getpkg("ObjConstPkg").constraints["objc"]["1"].lb = 0.9*mdlutl.pkgmgr.getpkg("ObjConstPkg").constraints["objc"]["1"].lb
            solution = mdlutl.model.optimize()
    output[modelid]["minimum probability objective"] = solution.objective_value
    
    with open(util.output_dir+"/"+modelid+"-final.lp", "w") as out:
        out.write(str(mdlutl.model.solver))

    records[3][modelid] = output[modelid]["minimum probability objective"]
    output[modelid]["solution"] = {}
    for rxn in mdlutl.model.reactions:
        if rxn.id not in rxn_record_hash:
            rxn_record_hash[rxn.id] = {
                "id":rxn.id,
            }
            for mdl in model_list:
                rxn_record_hash[rxn.id][mdl] = None
            records.append(rxn_record_hash[rxn.id])
        rxn_record_hash[rxn.id][modelid] = solution.fluxes[rxn.id]
        output[modelid]["solution"][rxn.id] = solution.fluxes[rxn.id]
    util.save("CommunityFluxSolution2", output)
    df = DataFrame.from_records(records)
    df.to_csv(util.output_dir+"/CommunityFluxSolution2.csv")